In [ ]:
import pandas as pd
import warnings
from sklearn.exceptions import ConvergenceWarning
import numpy as np
import os
import pickle
import math
import random
THRESHOLD = 10

warnings.filterwarnings('ignore', category=ConvergenceWarning)

class RegretAlgorithm:

    def __init__(self, n_arms, counts=None, emp_means=None):
        self.counts = counts if counts else [0 for col in range(n_arms)]
        self.emp_means = emp_means if emp_means else [0.0 for col in range(n_arms)]
        self.ranking = []  
        return
    def __str__(self):
        return None

    def reset(self, n_arms):
        self.counts = [0 for col in range(n_arms)]
        self.emp_means = [0.0 for col in range(n_arms)]
        self.ranking = []
        return

    def select_next_arm(self):
        pass

    def update(self, chosen_arm, reward):
        self.counts[chosen_arm] = self.counts[chosen_arm] + 1
        n = self.counts[chosen_arm]
        value = self.emp_means[chosen_arm]
        new_value = ((n - 1) / float(n)) * value + (1 / float(n)) * reward
        self.emp_means[chosen_arm] = new_value
        self.ranking = list(np.arange(len(self.counts))[np.argsort(self.counts)])
        return

class UCB1(RegretAlgorithm):
    def __init__(self, n_arms, counts=None, emp_means=None, ucbs=None, batch=False):
        RegretAlgorithm.__init__(self, n_arms, counts, emp_means)
        self.ucbs = ucbs if ucbs else [0.0 for col in range(n_arms)]  
        self.batch = batch
        return

    def __str__(self):
        return 'ucb1'

    def reset(self, n_arms):
        RegretAlgorithm.reset(self, n_arms)

        self.ucbs = [0.0 for col in range(n_arms)]
        return
    def select_next_arm(self):
        if not self.batch:

            if 0 in self.counts:  
                for arm in range(len(self.counts)):
                    if self.counts[arm] == 0:
                        return arm
            else:  
                return np.argmax(self.ucbs)
        else:
            return np.argmax(self.ucbs)

    def update(self, chosen_arm, reward):
        RegretAlgorithm.update(self, chosen_arm, reward)
        bonuses = [math.sqrt((2 * math.log(sum(self.counts) + 1)) / float(self.counts[arm] + 1e-7)) for arm in range(len(self.counts))]
        self.ucbs = [e + b for e, b in zip(self.emp_means, bonuses)]

        return
    
from scipy.stats import beta, norm      
class BayesUCBBeta(UCB1):

    def __init__(self, n_arms, counts=None, emp_means=None, ucbs=None, alphas=None, betas=None, c=2, batch=False):
        UCB1.__init__(self, n_arms, counts, emp_means, ucbs, batch)
        self.alphas = alphas if alphas else [1.0 for col in range(n_arms)]
        self.betas = betas if betas else [1.0 for col in range(n_arms)]
        self.c = c  
        return

    def __str__(self):
        return f'bayes_ucb_beta_c={self.c}'

    def reset(self, n_arms):
        UCB1.reset(self, n_arms)
        self.alphas = [1.0 for col in range(n_arms)]
        self.betas = [1.0 for col in range(n_arms)]
        return

    def update(self, chosen_arm, reward):
        RegretAlgorithm.update(self, chosen_arm, reward)

        self.alphas[chosen_arm] = self.alphas[chosen_arm] + reward
        self.betas[chosen_arm] = self.betas[chosen_arm] + (1-reward)

        means = [a/(a+b) for a, b in zip(self.alphas, self.betas)]
        stds = [self.c * beta.std(a, b) for a, b in zip(self.alphas, self.betas)]
        self.ucbs = [m + s for m, s in zip(means, stds)]
        
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
def create_model_ohe(save_dir="model/"):
    os.makedirs(save_dir, exist_ok=True)  
    kernel = C(1.0, (1e-7, 1e7)) * RBF(1.0, (1e-8, 1e8))
    model = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=20)
    model_path = os.path.join(save_dir, f"model.pkl")
    with open(model_path, "wb") as f:
        pickle.dump(model, f)
    return

from sklearn.tree import DecisionTreeRegressor
def create_model_dcr(save_dir="model/"):
    os.makedirs(save_dir, exist_ok=True)  
    model = DecisionTreeRegressor()
    model_path = os.path.join(save_dir, f"model.pkl")
    with open(model_path, "wb") as f:
        pickle.dump(model, f)
    return

from sklearn.svm import SVR
def create_model_svm(save_dir="model/"):
    os.makedirs(save_dir, exist_ok=True)  
    model = SVR()
    model_path = os.path.join(save_dir, f"model.pkl")
    with open(model_path, "wb") as f:
        pickle.dump(model, f)
    return

from sklearn.ensemble import RandomForestRegressor
def create_model_rmr(save_dir="model/"):
    os.makedirs(save_dir, exist_ok=True)  

    model = RandomForestRegressor()
    model_path = os.path.join(save_dir, f"model.pkl")
    with open(model_path, "wb") as f:
        pickle.dump(model, f)
    return

from sklearn.neighbors import KNeighborsRegressor
def create_model_knn(save_dir="model/"):
    os.makedirs(save_dir, exist_ok=True)  
    model = KNeighborsRegressor(n_neighbors=1)
    model_path = os.path.join(save_dir, f"model.pkl")
    with open(model_path, "wb") as f:
        pickle.dump(model, f)
    return

def load_model_ohe(save_dir="model/"):
    model_path = os.path.join(save_dir, "model.pkl")
    with open(model_path, "rb") as f:
        model = pickle.load(f)
    return model

def save_model_ohe(model, save_dir="model/"):
    model_path = os.path.join(save_dir, "model.pkl")
    with open(model_path, "wb") as f:
        pickle.dump(model, f)
    return

def fit_model_ohe(ohe_dict, substrate, record_data, save_dir="model/"):
    model = load_model_ohe(save_dir)
    X = []
    y = []    
    for condition, substrates in record_data.items():
        value = substrates[substrate]
        if value is not None :
            arm_ohe = ohe_dict.get((substrate, condition))
            if arm_ohe is not None:
                X.append(arm_ohe)
                y.append(value)                
    if len(X) == 0 or len(y) == 0:
        raise ValueError(f"No valid data found for substrate '{substrate}'.")
    X = np.array(X)
    y = np.array(y)
    model.fit(X, y)
    save_model_ohe(model,save_dir)
    return model 

def select_substrate_ohe(arm, Substrate_list, ohe_dict, record_data,save_dir="model/"):

    untrained_substrates = []
    for substrate in Substrate_list:
        if all(record_data[condition].get(substrate) is None for condition in record_data):
            untrained_substrates.append(substrate)
    if untrained_substrates:
        selected_substrate = random.choice(untrained_substrates) 
        return selected_substrate
    else:
        predict_substrates = []
        model=load_model_ohe(save_dir)
        for substrate in Substrate_list:
            X=ohe_dict[(substrate, arm)].reshape(1, -1)
            y_pred= model.predict(X)
            predict_substrates.append((substrate, y_pred[0])) 
        available_substrates = [substrate for substrate, _ in predict_substrates if record_data[arm].get(substrate) is None]
        if available_substrates:
            available_predictions = [(substrate, pred) for substrate, pred in predict_substrates if substrate in available_substrates]
            value = min(available_predictions, key=lambda x: x[1])[1]
            min_candidates = [item for item in available_predictions if item[1] == value]
            selected_substrate = random.choice(min_candidates)[0]
            return selected_substrate 
    return None

def generate_mordred(Substrate_list, arms_list):
    
    substrate_mordred = pd.read_csv(r'C:\Users\Administrator\Desktop\supply data\Ni-catalyzed borylation\electrophile_id_Mordred.csv')
    arms_mordred = pd.read_csv(r'C:\Users\Administrator\Desktop\supply data\Ni-catalyzed borylation\ligand_id_Mordred.csv')
    combinations = [(s, a) for s in Substrate_list for a in arms_list]
    mordred_dict = {}
    for s, a in combinations:
        substrate_code = substrate_mordred[substrate_mordred['id']==s].iloc[:,1:].values[0]
        arm_code = arms_mordred[arms_mordred['id']==a].iloc[:,1:].values[0]
        code = np.concatenate((substrate_code, arm_code))
        mordred_dict[(s, a)] = code
    return mordred_dict

def generate_CM(Substrate_list, arms_list):
    
    substrate_mordred = pd.read_csv(r'C:\Users\Administrator\Desktop\supply data\Ni-catalyzed borylation\electrophile_id_CM.csv')
    arms_mordred = pd.read_csv(r'C:\Users\Administrator\Desktop\supply data\Ni-catalyzed borylation\ligand_id_CM.csv')
    combinations = [(s, a) for s in Substrate_list for a in arms_list]
    mordred_dict = {}
    for s, a in combinations:
        substrate_code = substrate_mordred[substrate_mordred['id']==s].iloc[:,1:].values[0]
        arm_code = arms_mordred[arms_mordred['id']==a].iloc[:,1:].values[0]
        code = np.concatenate((substrate_code, arm_code))
        mordred_dict[(s, a)] = code
    return mordred_dict

def generate_ohe(Substrate_list, arms_list):

    combinations = [(s, a) for s in Substrate_list for a in arms_list]
    ohe_dict = {}
    for s, a in combinations:
        ohe = np.zeros(len(Substrate_list) * len(arms_list))
        index = Substrate_list.index(s) * len(arms_list) + arms_list.index(a)
        ohe[index] = 1
        ohe_dict[(s, a)] = ohe
    return ohe_dict

def generate_morgan(Substrate_list, arms_list): 
    substrate_morgan = pd.read_csv(r'C:\Users\Administrator\Desktop\supply data\Ni-catalyzed borylation\electrophile_id_Morgan.csv')
    arms_morgan = pd.read_csv(r'C:\Users\Administrator\Desktop\supply data\Ni-catalyzed borylation\ligand_id_Morgan.csv')
    combinations = [(s, a) for s in Substrate_list for a in arms_list]
    morgan_dict = {}
    for s, a in combinations:
        substrate_code = substrate_morgan[substrate_morgan['id']==s].iloc[:,1:].values[0]
        arm_code = arms_morgan[arms_morgan['id']==a].iloc[:,1:].values[0]
        code = np.concatenate((substrate_code, arm_code))
        morgan_dict[(s, a)] = code
    return morgan_dict


def generate_EI(Substrate_list, arms_list):
    substrate_mordred = pd.read_csv(r'C:\Users\Administrator\Desktop\supply data\Ni-catalyzed borylation\electrophile_id_EI.csv')
    arms_mordred = pd.read_csv(r'C:\Users\Administrator\Desktop\supply data\Ni-catalyzed borylation\ligand_id_EI.csv')
    combinations = [(s, a) for s in Substrate_list for a in arms_list]
    
    mordred_dict = {}
    for s, a in combinations:
        substrate_code = substrate_mordred[substrate_mordred['id']==s].iloc[:,1:].values[0]
        arm_code = arms_mordred[arms_mordred['id']==a].iloc[:,1:].values[0]
        code = np.concatenate((substrate_code, arm_code))
        mordred_dict[(s, a)] = code
    return mordred_dict

def get_field(data,arm,substrate):
    field=data[arm][substrate]
    return field

def recode_experiment(record_data,arm,substrate,field):
    record_data[arm][substrate]=field
    return record_data

def run_simulation(model,enconding,num_sims, horizons, arms_list, Substrate_list, data):

    results = []

    if enconding == 'mordred':
        ohe_dict = generate_mordred(Substrate_list, arms_list)
    elif enconding == 'EI':

        ohe_dict = generate_EI(Substrate_list, arms_list)
    elif enconding == 'CM':
    
        ohe_dict = generate_CM(Substrate_list, arms_list)
    elif enconding == 'One-hot':
  
        ohe_dict = generate_ohe(Substrate_list, arms_list)
    elif enconding == 'Morgan':
 
        ohe_dict = generate_morgan(Substrate_list, arms_list)
    else:
        raise ValueError("Unknown encoding type")

    for sim in range(num_sims):


        save_dir="model/"
        
        if os.path.exists(save_dir):
            for f in os.listdir(save_dir):
                os.remove(os.path.join(save_dir, f))
                print('model delete success')
                
        if model=='Gaussian_process':
            create_model_ohe(save_dir)

        elif model=='decision_tree':
            create_model_dcr(save_dir)

        elif model=='svm':
            create_model_svm(save_dir)

        elif model=='random_forest':
            create_model_rmr(save_dir)

        elif model=='Knn':
            create_model_knn(save_dir)

        record_data = {condition: {substrate: None for substrate in Substrate_list} for condition in arms_list}
        algo=BayesUCBBeta(len(arms_list))


        algo.reset(len(arms_list))

        history = []

        for horizon in range(horizons):
            arm = algo.select_next_arm()

            condition = arms_list[arm]

            substrate = select_substrate_ohe(condition, Substrate_list, ohe_dict, record_data, save_dir="model/")
            
            if substrate is None:
                threshold = 0  
                while substrate is None:
                    if threshold > THRESHOLD:  
                        substrate = random.choice(Substrate_list)
                        break
                    new_arm = algo.select_next_arm()
                    new_condition = arms_list[new_arm]
                    new_substrate = select_substrate_ohe(new_condition, Substrate_list, ohe_dict, record_data, save_dir="model/")
                    if new_arm == arm:
                        threshold += 1
                        continue  
                    else:
                        arm = new_arm
                        condition = new_condition
                        substrate = new_substrate
                        break  
                
            field = get_field(data, condition, substrate)
            record_data = recode_experiment(record_data, condition, substrate, field)
            history.append({
                'sim': sim + 1,  
                'horizon': horizon + 1,  
                'condition': condition,
                'substrate': substrate,
                'yield': field
            })
            algo.update(arm, field)
            fit_model_ohe(ohe_dict, substrate, record_data, save_dir="model/")

        results.extend(history)
        print(f"Simulation {sim + 1} completed.")
    return results

def data_anlyse():

    data_file=pd.read_csv(r'C:\Users\Administrator\Desktop\supply data\Ni-catalyzed borylation\inchi.csv')
    data_file=data_file.iloc[:,1:]
    Substrate_list=list(data_file['electrophile_name'].unique())
    arms_list=list(data_file['ligand_name'].unique())
    data_file['yield'] = data_file['yield'].apply(lambda x: 0 if x<50 else 1)
    data={}
    for i in arms_list:
        data[i]={}
        for j in Substrate_list:
            data[i][j]=data_file[(data_file['ligand_name']==i)&(data_file['electrophile_name']==j)]['yield'].values[0]
    return data,arms_list,Substrate_list

if __name__ == '__main__':
    data,arms_list,Substrate_list=data_anlyse()
    num=500
    encondings=['mordred','One-hot','CM','EI','Morgan']
    models=['svm','decision_tree','Gaussian_process','random_forest','Knn']

    for model in models:
        for enconding in encondings:
            history = run_simulation(
                model=model,
                enconding=enconding,
                num_sims=num,
                horizons=100,
                arms_list=arms_list,
                Substrate_list=Substrate_list,
                data=data,
            )
            history_df = pd.DataFrame(history)
            path=r'C:\Users\Administrator\Desktop\supply data\Basic experiment\Ni-catalyzed borylation\results'
            history_df.to_csv(path+f'\{model}_history_{num}_{enconding}_beta.csv', index=False)

model delete success
Simulation 1 completed.
model delete success
Simulation 2 completed.
model delete success
Simulation 3 completed.
model delete success
Simulation 4 completed.
model delete success
Simulation 5 completed.
model delete success
Simulation 6 completed.
model delete success
Simulation 7 completed.
model delete success
Simulation 8 completed.
model delete success
Simulation 9 completed.
model delete success
Simulation 10 completed.
model delete success
Simulation 11 completed.
model delete success
Simulation 12 completed.
model delete success
Simulation 13 completed.
model delete success
Simulation 14 completed.
model delete success
Simulation 15 completed.
model delete success
Simulation 16 completed.
model delete success
Simulation 17 completed.
model delete success
Simulation 18 completed.
model delete success
Simulation 19 completed.
model delete success
Simulation 20 completed.
model delete success
Simulation 21 completed.
model delete success
Simulation 22 complete